In [ ]:
#pip install faiss-cpu

In [ ]:
from openai import OpenAI
from google.colab import userdata
import base64
from PIL import Image
import io
import faiss
import numpy as np
import google.generativeai as genai
import json
import os

In [ ]:

client = OpenAI(
base_url="https://generativelanguage.googleapis.com/v1beta/openai",
api_key=userdata.get('GOOGLE_API_KEY')
)

In [ ]:
def encode_image_to_base64(image_path):
  with open(image_path, "rb") as img_file:
    return base64.b64encode(img_file.read()).decode("utf-8")

image_path = "/content/IMG-20250619-WA0070.jpg" # Path to your dog image
image_base64 = encode_image_to_base64(image_path)

In [ ]:
prompt = """
You are an expert in dog breed recognition and behavior analysis. Given the image of a dog, analyze it and provide the following details in a consistent, structured bullet-point format (one line per attribute, no extra commentary):

Breed: (e.g., Labrador Retriever, German Shepherd, Mixed, Unknown)

Color & Markings: (e.g., Golden with white chest, Black and tan, Spotted)

Fur Type: (e.g., short, long, curly, wiry)

Size: (small / medium / large)

Ear Type: (e.g., floppy, erect, semi-erect)

Tail Type: (e.g., long, curled, bushy, short)

Return only the above list using the exact same order and labels. Use plain English, and write “Unknown” if a feature is not visible or determinable.
"""

In [ ]:
response = client.chat.completions.create(
model="gemini-1.5-flash",
messages=[
{
"role": "user",
"content": [
{"type": "text", "text": prompt},
{
"type": "image_url",
"image_url": {
"url": f"data:image/jpeg;base64,{image_base64}"
}
}
]
}
],
max_tokens=512,
temperature=0.7
)

In [ ]:
print("Dog Identity Description:\n")
desc = response.choices[0].message.content
print(desc)

Dog Identity Description:

Breed: Mixed

Color & Markings: Light Brown

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown


In [ ]:
response = genai.embed_content(
model="models/embedding-001", # Gemini's text embedding model
content=desc.strip(),
task_type="retrieval_document" # You can also use "retrieval_query"
)
print(desc.strip())

Breed: Mixed

Color & Markings: Light Brown

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown


In [ ]:
embedding = np.array(response["embedding"], dtype="float32")
print(len(embedding))

768


In [ ]:
INDEX_PATH = "dog_vectors.index"
METADATA_PATH = "dog_metadata.json"
dimension = len(embedding)
if os.path.exists(INDEX_PATH):
  index = faiss.read_index(INDEX_PATH)
  print("yes")
else:
  index = faiss.IndexFlatL2(dimension)
  print("no")

index.add(np.array([embedding]))  # ✅ first add embedding
faiss.write_index(index, INDEX_PATH)  # ✅ then save it to disk


yes


In [ ]:
dimension = len(embedding)
import random
import string

def generate_dog_id(length=6):
  characters = string.ascii_uppercase + string.digits
  return ''.join(random.choices(characters, k=length))
dog_id = generate_dog_id()
index.add(np.array([embedding]))      # Add current embedding

In [ ]:
if os.path.exists(METADATA_PATH):
  with open(METADATA_PATH, "r") as f:
    metadata_list = json.load(f)
    print("yes")
else:
  metadata_list = []

metadata_list.append({
"dog_id": dog_id,
"image_path": image_path,
"description": desc.strip()
})

yes


In [ ]:
with open(METADATA_PATH, "w") as f:
  json.dump(metadata_list, f, indent=2)

print(f"✅ Successfully stored dog ID: {dog_id} with image: {image_path}")

✅ Successfully stored dog ID: NK72UU with image: /content/IMG-20250619-WA0070.jpg


In [ ]:
query_desc = """
Breed: Mixed
Color & Markings: Light Brown/Fawn
Fur Type: Short
Size: Medium
Ear Type: Semi-erect
Tail Type: Unknown
"""

In [ ]:
query_embedding = genai.embed_content(
model="models/embedding-001",
content=query_desc.strip(),
task_type="retrieval_query"
)["embedding"]
query_embedding = np.array(query_embedding, dtype="float32").reshape(1, -1)

In [ ]:
top_k = 5
distances, indices = index.search(query_embedding, top_k)

In [ ]:
print(f"\n🔍 Top {top_k} Matches:")
for i, idx in enumerate(indices[0]):
  if idx >= len(metadata_list):
    continue # skip invalid index
  metadata = metadata_list[idx]
  similarity = 1 - (distances[0][i] / 2) # Convert L2 distance to similarity approximation
  similarity = round(similarity * 100, 2)
  print(f"\nResult {i+1}")
  print(f"✅ Dog ID: {metadata['dog_id']}")
  print(f"🖼️ Image Path: {metadata['image_path']}")
  print(f"📋 Description:\n{metadata['description']}")
  print(f"📊 Similarity: {similarity}%")

print(index.ntotal)


🔍 Top 5 Matches:

Result 1
✅ Dog ID: U0SULA
🖼️ Image Path: /content/IMG-20250619-WA0009.jpg
📋 Description:
Breed: Mixed

Color & Markings: Light Brown/Tan

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
📊 Similarity: 81.30000305175781%

Result 2
✅ Dog ID: MRLBTE
🖼️ Image Path: /content/IMG-20250619-WA0099.jpg
📋 Description:
Breed: Mixed

Color & Markings: Light tan/fawn

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
📊 Similarity: 81.08000183105469%

Result 3
✅ Dog ID: 9SSQX9
🖼️ Image Path: /content/IMG-20250619-WA0011.jpg
📋 Description:
Breed: Mixed

Color & Markings: Light Brown

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
📊 Similarity: 81.08000183105469%

Result 4
✅ Dog ID: 3SG6WI
🖼️ Image Path: /content/IMG-20250619-WA0013.jpg
📋 Description:
Breed: Mixed

Color & Markings: Light Brown

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
📊 Similarity: 81.08000183105469%

Result 5
✅ Dog

## **Approach 2**

In [ ]:
print(index.ntotal)

30


In [ ]:
for i, entry in enumerate(metadata_list):
  print(f"🆔 ID: {entry['dog_id']}")
  print(f"🖼️ Image: {entry['image_path']}")
  print(f"📄 Description:\n{entry['description']}")

🆔 ID: RMAWFT
🖼️ Image: /content/IMG-20250619-WA0031.jpg
📄 Description:
Breed: Mixed

Color & Markings: Black

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
🆔 ID: 9XZI74
🖼️ Image: /content/IMG-20250619-WA0008.jpg
📄 Description:
Breed: Mixed

Color & Markings: Black and white

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Long
🆔 ID: U0SULA
🖼️ Image: /content/IMG-20250619-WA0009.jpg
📄 Description:
Breed: Mixed

Color & Markings: Light Brown/Tan

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
🆔 ID: WJPM0N
🖼️ Image: /content/IMG-20250619-WA0010.jpg
📄 Description:
Breed: Mixed

Color & Markings: Black and white

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Long
🆔 ID: 9SSQX9
🖼️ Image: /content/IMG-20250619-WA0011.jpg
📄 Description:
Breed: Mixed

Color & Markings: Light Brown

Fur Type: Short

Size: Medium

Ear Type: Semi-erect

Tail Type: Unknown
🆔 ID: PIF7DE
🖼️ Image: /content/IMG-20250619-WA0012.jpg
📄 De